# Agate Agony: Converting Date Columns to Text after ingestion

as a pandas stan, agate is somewhat inscrutable to me. I need to cast Date columns to Text because... reasons.

The challenge is that it has to happen *after* the csv is loaded into memory as an Agate table

In [91]:
import agate
import os
from loguru import logger

In [82]:
os.chdir('/Users/anders.swanson/repos/jaffle_shop/')
fname='data/raw_orders.csv'

## the data
below you can see there are two date columns: `order` and `date` that are unquoted strings

In [83]:
! cat data/raw_orders.csv

id,user_id,order_date,status
1,1,2018-01-01,returned
2,3,2018-01-02,completed
3,94,2018-01-04,completed
4,50,2018-01-05,completed
5,64,2018-01-05,completed
6,54,2018-01-07,completed
7,88,2018-01-09,completed
8,2,2018-01-11,returned
9,53,2018-01-12,completed
10,7,2018-01-14,completed
11,99,2018-01-14,completed
12,59,2018-01-15,completed
13,84,2018-01-17,completed
14,40,2018-01-17,returned
15,25,2018-01-17,completed
16,39,2018-01-18,completed
17,71,2018-01-18,completed
18,64,2018-01-20,returned
19,54,2018-01-22,completed
20,20,2018-01-23,completed
21,71,2018-01-23,completed
22,86,2018-01-24,completed
23,22,2018-01-26,return_pending
24,3,2018-01-27,completed
25,51,2018-01-28,completed
26,32,2018-01-28,completed
27,94,2018-01-29,completed
28,8,2018-01-29,completed
29,57,2018-01-31,completed
30,69,2018-02-02,completed
31,16,2018-02-02,completed
32,28,2018-02-04,completed
33,42,2018-02-04,completed
34,38,2018-02-06,completed
35,80,2018-02-08,completed
36,85,2018-02-10,completed
37,1,2018-02-

## wishful thinking

If I had control of how the agate table was loaded into memory, the job is easy

In [106]:
tester = agate.TypeTester(types=[agate.Text()])
# orders_txt = agate.Table.from_csv(fname, column_types=tester)
orders_txt = agate.Table.from_csv(fname, column_types={'order_date': agate.Text()})

orders_txt.print_structure(),orders_txt.print_table()

| column     | data_type |
| ---------- | --------- |
| id         | Number    |
| user_id    | Number    |
| order_date | Text      |
| status     | Text      |
| id | user_id | order_date | status    |
| -- | ------- | ---------- | --------- |
|  1 |       1 | 2018-01-01 | returned  |
|  2 |       3 | 2018-01-02 | completed |
|  3 |      94 | 2018-01-04 | completed |
|  4 |      50 | 2018-01-05 | completed |
|  5 |      64 | 2018-01-05 | completed |
|  6 |      54 | 2018-01-07 | completed |
|  7 |      88 | 2018-01-09 | completed |
|  8 |       2 | 2018-01-11 | returned  |
|  9 |      53 | 2018-01-12 | completed |
| 10 |       7 | 2018-01-14 | completed |
| 11 |      99 | 2018-01-14 | completed |
| 12 |      59 | 2018-01-15 | completed |
| 13 |      84 | 2018-01-17 | completed |
| 14 |      40 | 2018-01-17 | returned  |
| 15 |      25 | 2018-01-17 | completed |
| 16 |      39 | 2018-01-18 | completed |
| 17 |      71 | 2018-01-18 | completed |
| 18 |      64 | 2018-01-20 | returned  

(None, None)

## the given object

instead we are forced to work with the the auto-inferred column types which are correct, but whatever

In [85]:
orders = agate.Table.from_csv(fname)

orders.print_structure(),orders.print_table()

| column     | data_type |
| ---------- | --------- |
| id         | Number    |
| user_id    | Number    |
| order_date | Date      |
| status     | Text      |
| id | user_id | order_date | status    |
| -- | ------- | ---------- | --------- |
|  1 |       1 | 2018-01-01 | returned  |
|  2 |       3 | 2018-01-02 | completed |
|  3 |      94 | 2018-01-04 | completed |
|  4 |      50 | 2018-01-05 | completed |
|  5 |      64 | 2018-01-05 | completed |
|  6 |      54 | 2018-01-07 | completed |
|  7 |      88 | 2018-01-09 | completed |
|  8 |       2 | 2018-01-11 | returned  |
|  9 |      53 | 2018-01-12 | completed |
| 10 |       7 | 2018-01-14 | completed |
| 11 |      99 | 2018-01-14 | completed |
| 12 |      59 | 2018-01-15 | completed |
| 13 |      84 | 2018-01-17 | completed |
| 14 |      40 | 2018-01-17 | returned  |
| 15 |      25 | 2018-01-17 | completed |
| 16 |      39 | 2018-01-18 | completed |
| 17 |      71 | 2018-01-18 | completed |
| 18 |      64 | 2018-01-20 | returned  

(None, None)

## solution
the solution is to create the following
- a function to generate:
    - a column-specifc `agate.Formula`: i.e. `agate`-flavored lambda function that in this case does nothing except ensure it's output will be cast to `agate.TypeText`
- a list comprehension the return a list of tuples of length two for each Date column consisting of:
    - the column name, and
    - the column's special `agate.Formula`
- a `.compute()` call to the table that applies the `Formulas` to each column, overwriting the original columns with text versions

In [104]:


def convert_agate_date_cols_to_text_cols3(agate_table) -> agate.Table:
    def cast_col_to_text(col_name):
            return agate.Formula(agate.Text(), lambda r: r[col_name])
    
    computations = [(
        col.name,
        cast_col_to_text(col.name)
        ) for col in agate_table.columns if 'Date' in str(col.data_type)]

    return agate_table.compute(computations,replace=True)

orders_new = convert_agate_date_cols_to_text_cols3(orders)
orders_new.print_structure(),orders_new.print_table()

| column     | data_type |
| ---------- | --------- |
| id         | Number    |
| user_id    | Number    |
| order_date | Text      |
| status     | Text      |
| id | user_id | order_date | status    |
| -- | ------- | ---------- | --------- |
|  1 |       1 | 2018-01-01 | returned  |
|  2 |       3 | 2018-01-02 | completed |
|  3 |      94 | 2018-01-04 | completed |
|  4 |      50 | 2018-01-05 | completed |
|  5 |      64 | 2018-01-05 | completed |
|  6 |      54 | 2018-01-07 | completed |
|  7 |      88 | 2018-01-09 | completed |
|  8 |       2 | 2018-01-11 | returned  |
|  9 |      53 | 2018-01-12 | completed |
| 10 |       7 | 2018-01-14 | completed |
| 11 |      99 | 2018-01-14 | completed |
| 12 |      59 | 2018-01-15 | completed |
| 13 |      84 | 2018-01-17 | completed |
| 14 |      40 | 2018-01-17 | returned  |
| 15 |      25 | 2018-01-17 | completed |
| 16 |      39 | 2018-01-18 | completed |
| 17 |      71 | 2018-01-18 | completed |
| 18 |      64 | 2018-01-20 | returned  

(None, None)

In [101]:
def convert_agate_date_cols_to_text_cols2(agate_table) -> agate.Table:
        col_list = []
        for col in agate_table.columns:
            if 'Date' in str(col.data_type) or 'Text' in str(col.data_type):
                logger.info(f"col to transform: {col.name}")
                col_list.append(col.name)

        def cast_col_to_text(col_name):
            return agate.Formula(agate.Text(), lambda r: r[col_name])

        computations = []
        for col_name in col_list:
            tmp = (
                col_name,
                agate.Formula(agate.Text(), lambda r: r[col.name])
                )
            
            logger.info(f"added tup: {tmp}")
            
            computations.append(tmp)
        logger.info(f"len of comps: {len(computations)}")

        new_table = agate_table.compute(computations,replace=True)

        logger.info(f"new table struct: {new_table.print_structure()}")
        logger.info(f"new table head  : {new_table.limit(10).print_table()}")
        
        return new_table
    
orders_new = convert_agate_date_cols_to_text_cols2(orders)
orders_new.print_structure(),orders_new.print_table()

2021-08-06 17:25:54.367 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:5 - col to transform: order_date
2021-08-06 17:25:54.369 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:5 - col to transform: status
2021-08-06 17:25:54.370 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:15 - added tup: ('order_date', <agate.computations.formula.Formula object at 0x7fb6ec4e4e20>)
2021-08-06 17:25:54.371 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:15 - added tup: ('status', <agate.computations.formula.Formula object at 0x7fb6ec208400>)
2021-08-06 17:25:54.372 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:18 - len of comps: 2
2021-08-06 17:25:54.374 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:22 - new table struct: None
2021-08-06 17:25:54.377 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:23 - new table head  : None


| column     | data_type |
| ---------- | --------- |
| id         | Number    |
| user_id    | Number    |
| order_date | Text      |
| status     | Text      |
| id | user_id | order_date | status    |
| -- | ------- | ---------- | --------- |
|  1 |       1 | returned   | returned  |
|  2 |       3 | completed  | completed |
|  3 |      94 | completed  | completed |
|  4 |      50 | completed  | completed |
|  5 |      64 | completed  | completed |
|  6 |      54 | completed  | completed |
|  7 |      88 | completed  | completed |
|  8 |       2 | returned   | returned  |
|  9 |      53 | completed  | completed |
| 10 |       7 | completed  | completed |
| column     | data_type |
| ---------- | --------- |
| id         | Number    |
| user_id    | Number    |
| order_date | Text      |
| status     | Text      |
| id | user_id | order_date | status    |
| -- | ------- | ---------- | --------- |
|  1 |       1 | returned   | returned  |
|  2 |       3 | completed  | completed |
|  3

(None, None)

2021-08-06 17:25:44.115 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:5 - col to transform: order_date
2021-08-06 17:25:44.117 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:5 - col to transform: status
2021-08-06 17:25:44.118 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:15 - added tup: ('order_date', <agate.computations.formula.Formula object at 0x7fb6ecadc9a0>)
2021-08-06 17:25:44.120 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:15 - added tup: ('status', <agate.computations.formula.Formula object at 0x7fb6ecadc430>)
2021-08-06 17:25:44.121 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:18 - len of comps: 2
2021-08-06 17:25:44.126 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:22 - new table struct: None
2021-08-06 17:25:44.131 | INFO     | __main__:convert_agate_date_cols_to_text_cols2:23 - new table head  : None


| column     | data_type |
| ---------- | --------- |
| id         | Number    |
| user_id    | Number    |
| order_date | Text      |
| status     | Text      |
| id | user_id | order_date | status    |
| -- | ------- | ---------- | --------- |
|  1 |       1 | returned   | returned  |
|  2 |       3 | completed  | completed |
|  3 |      94 | completed  | completed |
|  4 |      50 | completed  | completed |
|  5 |      64 | completed  | completed |
|  6 |      54 | completed  | completed |
|  7 |      88 | completed  | completed |
|  8 |       2 | returned   | returned  |
|  9 |      53 | completed  | completed |
| 10 |       7 | completed  | completed |
| column     | data_type |
| ---------- | --------- |
| id         | Number    |
| user_id    | Number    |
| order_date | Text      |
| status     | Text      |
| id | user_id | order_date | status    |
| -- | ------- | ---------- | --------- |
|  1 |       1 | returned   | returned  |
|  2 |       3 | completed  | completed |
|  3

(None, None)

In [86]:
def convert_agate_date_cols_to_text_cols(agate_table) -> agate.Table:

    computations = [(
        col.name,
        agate.Formula(agate.Text(), lambda r: r[col.name], cast=True)
        ) for col in agate_table.columns if 'Date' in str(col.data_type)]

    return agate_table.compute(computations,replace=True)

In [87]:
orders_new = convert_agate_date_cols_to_text_cols(orders)
orders_new.print_structure(),orders_new.print_table()

| column     | data_type |
| ---------- | --------- |
| id         | Number    |
| user_id    | Number    |
| order_date | Text      |
| status     | Text      |
| id | user_id | order_date | status    |
| -- | ------- | ---------- | --------- |
|  1 |       1 | returned   | returned  |
|  2 |       3 | completed  | completed |
|  3 |      94 | completed  | completed |
|  4 |      50 | completed  | completed |
|  5 |      64 | completed  | completed |
|  6 |      54 | completed  | completed |
|  7 |      88 | completed  | completed |
|  8 |       2 | returned   | returned  |
|  9 |      53 | completed  | completed |
| 10 |       7 | completed  | completed |
| 11 |      99 | completed  | completed |
| 12 |      59 | completed  | completed |
| 13 |      84 | completed  | completed |
| 14 |      40 | returned   | returned  |
| 15 |      25 | completed  | completed |
| 16 |      39 | completed  | completed |
| 17 |      71 | completed  | completed |
| 18 |      64 | returned   | returned  

(None, None)

In [98]:
def cast_col_to_text(col_name):
    return agate.Formula(agate.Text(), lambda r: r[col_name])

out = [(col.name, cast_col_to_text(col.name)) for col in orders.columns if 'Date' in str(col.data_type)]
orders_hack = orders.compute(out,replace=True)

orders_hack.print_structure(),orders_hack.print_table()

| column     | data_type |
| ---------- | --------- |
| id         | Number    |
| user_id    | Number    |
| order_date | Text      |
| status     | Text      |
| id | user_id | order_date | status    |
| -- | ------- | ---------- | --------- |
|  1 |       1 | 2018-01-01 | returned  |
|  2 |       3 | 2018-01-02 | completed |
|  3 |      94 | 2018-01-04 | completed |
|  4 |      50 | 2018-01-05 | completed |
|  5 |      64 | 2018-01-05 | completed |
|  6 |      54 | 2018-01-07 | completed |
|  7 |      88 | 2018-01-09 | completed |
|  8 |       2 | 2018-01-11 | returned  |
|  9 |      53 | 2018-01-12 | completed |
| 10 |       7 | 2018-01-14 | completed |
| 11 |      99 | 2018-01-14 | completed |
| 12 |      59 | 2018-01-15 | completed |
| 13 |      84 | 2018-01-17 | completed |
| 14 |      40 | 2018-01-17 | returned  |
| 15 |      25 | 2018-01-17 | completed |
| 16 |      39 | 2018-01-18 | completed |
| 17 |      71 | 2018-01-18 | completed |
| 18 |      64 | 2018-01-20 | returned  

(None, None)

In [79]:
def convert_agate_date_cols_to_text_cols(agate_table) -> agate.Table:

    computations = [(
        col.name,
        agate.Formula(agate.Text(), lambda r: r[col.name], cast=True)
        ) for col in agate_table.columns if 'Date' in str(col.data_type)]

    return agate_table.compute(computations,replace=True)